In [1]:
import arcpy
import pandas as pd
import psycopg2
from psycopg2 import sql
import numpy as np
import os
import folium

In [2]:
cwd = os.getcwd()

# Make it workspace

arcpy.env.workspace = os.path.join(cwd, '..', '..', 'data', 'QAQC.gdb')

arcpy.env.overwriteOutput = True # Overwrite layers is okay

save_path_csv = os.path.join(cwd, '..', '..', 'data')
save_path_gdb = os.path.join(cwd, '..', '..', 'data', 'QAQC.gdb')
historic_path = os.path.join(cwd, '..', '..', 'data', 'purpleair_historic.csv')
station_path = os.path.join(cwd, '..', '..', 'data', 'purpleair_stations.csv')
boundary_path = os.path.join(cwd, '..', '..', 'data', 'mpls_boundary.geojson')

In [3]:
output_fc = 'mpls_boundary'
output_fc_path = os.path.join(save_path_gdb, output_fc)

# Convert the GeoJSON file to a feature class
arcpy.JSONToFeatures_conversion(boundary_path, output_fc)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\mpls_boundary'>

In [4]:
df = pd.read_csv(historic_path)

# Convert the 'timestamp' column to a datetime object
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Filter the rows that have a timestamp of 4/1/23
date_filter = df['timestamp'].dt.date == pd.to_datetime('4/1/23').date()
filtered_df = df[date_filter]

filtered_df.to_csv('historic_april1.csv', index=False)
historic_filtered_path = os.path.join(cwd, '..', '..', 'data', 'historic_april1.csv')

In [5]:
# Load the first CSV file into a DataFrame
df1 = pd.read_csv(station_path)

# Load the second CSV file into a DataFrame
df2 = pd.read_csv(historic_filtered_path)

# Merge the two DataFrames together on a common column
merged_df = pd.merge(df1, df2, on='sensor_index')

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('merged_file.csv', index=False)
merged_path = os.path.join(cwd, '..', '..', 'data', 'merged_file.csv')

In [6]:
# Set output point feature class name and path
output_point_fc = "stations_XYTableToPoint"
output_point_path = os.path.join(save_path_gdb, output_point_fc)

# Specify x,y fields
x_field = "longitude"
y_field = "latitude"

# Use XYTableToPoint tool to create point features
station_points = arcpy.management.XYTableToPoint(merged_path, output_point_path, x_field, y_field)

In [7]:
'''
output_table = "joined_table"
arcpy.management.AddJoin(
    in_layer_or_view=station_points,
    in_field="sensor_index",
    join_table=historic_filtered_path,
    join_field="sensor_index",
    join_type="KEEP_ALL",
    index_join_fields="NO_INDEX_JOIN_FIELDS"
)

# Export the joined layer to a new table
historic_apr1 = arcpy.management.CopyRows("stations_XYTableToPoint", output_table)
'''

'\noutput_table = "joined_table"\narcpy.management.AddJoin(\n    in_layer_or_view=station_points,\n    in_field="sensor_index",\n    join_table=historic_filtered_path,\n    join_field="sensor_index",\n    join_type="KEEP_ALL",\n    index_join_fields="NO_INDEX_JOIN_FIELDS"\n)\n\n# Export the joined layer to a new table\nhistoric_apr1 = arcpy.management.CopyRows("stations_XYTableToPoint", output_table)\n'

## PM 2.5 Interpolation

### Kriging PM 2.5

In [38]:
out_kriging_raster = "Kriging_Apr1"
arcpy.ga.EmpiricalBayesianKriging(
    in_features="stations_XYTableToPoint",
    z_field="pm2_5",
    out_ga_layer="pm2_5_kriging_stats",
    out_raster=out_kriging_raster,
    cell_size=0.000932919999999967,
    transformation_type="NONE",
    max_local_points=100,
    overlap_factor=1,
    number_semivariograms=100,
    search_neighborhood="NBRTYPE=StandardCircular RADIUS=9.87111293434023E-02 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    output_type="PREDICTION",
    quantile_value=0.5,
    threshold_type="EXCEED",
    probability_threshold=None,
    semivariogram_model_type="POWER"
)

<Result 'pm2_5_kriging_stats'>

### IDW PM 2.5

In [42]:
out_idw_raster = "IDW_Apr1"
arcpy.ga.IDW(
    in_features="stations_XYTableToPoint",
    z_field="pm2_5",
    out_ga_layer="pm2_5_idw_stats",
    out_raster=out_idw_raster,
    cell_size=10,
    power=2,
    search_neighborhood="NBRTYPE=Standard S_MAJOR=9.87111293434023E-02 S_MINOR=9.87111293434023E-02 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    weight_field=None
)

<Result 'pm2_5_idw_stats'>

### Local Polynomial PM 2.5

In [44]:
out_poly_raster = "Poly_Apr1"
arcpy.ga.LocalPolynomialInterpolation(
    in_features="stations_XYTableToPoint",
    z_field="pm2_5",
    out_ga_layer="pm2_5_poly_stats",
    out_raster=out_poly_raster,
    cell_size=0.000932919999999967,
    power=1,
    search_neighborhood="NBRTYPE=Standard S_MAJOR=9.87111293434023E-02 S_MINOR=9.87111293434023E-02 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    kernel_function="EXPONENTIAL",
    bandwidth=None,
    use_condition_number="NO_USE_CONDITION_NUMBER",
    condition_number=None,
    weight_field=None,
    output_type="PREDICTION"
)

<Result 'pm2_5_poly_stats'>

## Temperature Interpolation

### Kriging Temperature

In [45]:
out_kriging_temp_raster = "Kriging_Temp_Apr1"
arcpy.ga.EmpiricalBayesianKriging(
    in_features="stations_XYTableToPoint",
    z_field="temperature",
    out_ga_layer="temp_kriging_stats",
    out_raster=out_kriging_temp_raster,
    cell_size=0.000932919999999967,
    transformation_type="NONE",
    max_local_points=100,
    overlap_factor=1,
    number_semivariograms=100,
    search_neighborhood="NBRTYPE=StandardCircular RADIUS=9.87111293434023E-02 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    output_type="PREDICTION",
    quantile_value=0.5,
    threshold_type="EXCEED",
    probability_threshold=None,
    semivariogram_model_type="POWER"
)

<Result 'temp_kriging_stats'>

### IDW Temp

In [46]:
out_idw_temp_raster = "IDW_Temp_Apr1"
arcpy.ga.IDW(
    in_features="stations_XYTableToPoint",
    z_field="temperature",
    out_ga_layer="temp_idw_stats",
    out_raster=out_idw_temp_raster,
    cell_size=0.000932919999999967,
    power=2,
    search_neighborhood="NBRTYPE=Standard S_MAJOR=9.87111293434023E-02 S_MINOR=9.87111293434023E-02 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    weight_field=None
)

<Result 'temp_idw_stats'>

### Local Polynomial Temp

In [47]:
out_poly_temp_raster = "Poly_temp_Apr1"
arcpy.ga.LocalPolynomialInterpolation(
    in_features="stations_XYTableToPoint",
    z_field="temperature",
    out_ga_layer="temp_poly_stats",
    out_raster=out_poly_temp_raster,
    cell_size=0.000932919999999967,
    power=1,
    search_neighborhood="NBRTYPE=Standard S_MAJOR=9.87111293434023E-02 S_MINOR=9.87111293434023E-02 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    kernel_function="EXPONENTIAL",
    bandwidth=None,
    use_condition_number="NO_USE_CONDITION_NUMBER",
    condition_number=None,
    weight_field=None,
    output_type="PREDICTION"
)

<Result 'temp_poly_stats'>

## Cross Validation

In [119]:
#creating an empty data frame to hold the rmse errors
rmse_df = pd.DataFrame(columns=['OBJECTID', 'latitude', 'longitude', "PM2_5_Kriging_Error", "PM2_5_IDW_Error", "PM2_5_Poly_Error", "Temp_Kriging_Error", "Temp_IDW_Error", "Temp_Poly_Error"])

In [126]:
rmse_df

,OBJECTID_x,latitude,longitude_x,PM2_5_Kriging_Error,PM2_5_IDW_Error,PM2_5_Poly_Error,Temp_Kriging_Error,Temp_IDW_Error,Temp_Poly_Error,OBJECTID_y,longitude_y,Error_x,Error_y
0,1,0.0,0.0,1.886304,NaN,NaN,NaN,NaN,NaN,1,0.0,2.092020,2.092020
1,1,0.0,0.0,1.886304,NaN,NaN,NaN,NaN,NaN,1,0.0,2.092020,3.066576
2,1,0.0,0.0,1.886304,NaN,NaN,NaN,NaN,NaN,1,0.0,2.092020,-27.152979
3,1,0.0,0.0,1.886304,NaN,NaN,NaN,NaN,NaN,1,0.0,2.092020,-1.163698
4,1,0.0,0.0,1.886304,NaN,NaN,NaN,NaN,NaN,1,0.0,2.092020,-0.204923
...,...,...,...,...,...,...,...,...,...,...,...,...,...
456528,77,0.0,0.0,2.442549,NaN,NaN,NaN,NaN,NaN,77,0.0,2.632748,0.690085
456529,77,0.0,0.0,2.442549,NaN,NaN,NaN,NaN,NaN,77,0.0,2.632748,2.953680
456530,77,0.0,0.0,2.442549,NaN,NaN,NaN,NaN,NaN,77,0.0,2.632748,3.110879
456531,77,0.0,0.0,2.442549,NaN,NaN,NaN,NaN,NaN,77,0.0,2.632748,2.396766


### Kriging PM 2.5

In [103]:
pm2_5_kriging_fc = 'pm2_5_kriging_cross_valid'
pm2_5_kriging_path = os.path.join(save_path_gdb, pm2_5_kriging_fc)
arcpy.ga.CrossValidation(
    in_geostat_layer="pm2_5_kriging_stats",
    out_point_feature_class=pm2_5_kriging_fc
)
pm2_5_kriging_out_fc = "PM2_5_Kriging_Join"
arcpy.analysis.SpatialJoin(
    target_features="stations_XYTableToPoint",
    join_features="pm2_5_kriging_cross_valid",
    out_feature_class=pm2_5_kriging_out_fc,
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_COMMON",
    field_mapping='OID_ "OID_" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.OID_,-1,-1;sensor_index "sensor_index" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.sensor_index,-1,-1;last_modified "last_modified" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.last_modified,-1,-1;date_created "date_created" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.date_created,-1,-1;last_seen "last_seen" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.last_seen,-1,-1;name "name" true true false 100 Text 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.name,0,100;location_type "location_type" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.location_type,-1,-1;firmware_version "firmware_version" true true false 30 Text 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.firmware_version,0,30;uptime "uptime" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.uptime,-1,-1;position_rating "position_rating" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.position_rating,-1,-1;latitude "latitude" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.latitude,-1,-1;longitude "longitude" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.longitude,-1,-1;altitude "altitude" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.altitude,-1,-1;channel_state "channel_state" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.channel_state,-1,-1;channel_flags "channel_flags" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.channel_flags,-1,-1;OID1 "OID_" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.OID_,-1,-1;sensor_index_1 "sensor_index" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.sensor_index,-1,-1;timestamp "timestamp" true true false 8 Date 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.timestamp,-1,-1;humidity "humidity" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.humidity,-1,-1;temperature "temperature" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.temperature,-1,-1;pressure "pressure" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.pressure,-1,-1;pm2_5 "pm2_5" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.pm2_5,-1,-1;Measured "Measured" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,Measured,-1,-1;Predicted "Predicted" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,Predicted,-1,-1;Error "Error" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,Error,-1,-1;StdError "Standard Error" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,StdError,-1,-1;Stdd_Error "Standardized Error" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,Stdd_Error,-1,-1;NormValue "Normal Value" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,NormValue,-1,-1;CRPS "Continuous Ranked Probability Score" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,CRPS,-1,-1;QuanVal "Validation Quantile" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,QuanVal,-1,-1;Interval90 "Inside 90 Percent Interval" true true false 255 Text 0 0,First,#,pm2_5_kriging_cross_valid,Interval90,0,255;Interval95 "Inside 95 Percent Interval" true true false 255 Text 0 0,First,#,pm2_5_kriging_cross_valid,Interval95,0,255;Source_ID "Source ID" true false false 4 Long 0 0,First,#,pm2_5_kriging_cross_valid,Source_ID,-1,-1;Included "Included" true true false 255 Text 0 0,First,#,pm2_5_kriging_cross_valid,Included,0,255',
    match_option="INTERSECT",
    search_radius=None,
    distance_field_name=""
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\pm2_5_kriging_cross_valid'>

In [120]:
input = pm2_5_kriging_out_fc
arr_temp_kriging = arcpy.da.TableToNumPyArray(input, ("OBJECTID", "latitude", "longitude", "Error"))
kriging_temp_arr = pd.DataFrame(arr_temp_kriging, columns=["OBJECTID", "latitude", "longitude", "Error"])
kriging_temp_err = kriging_temp_arr.rename(columns={"Error": "PM2_5_Kriging_Error"})
rmse_df = pd.concat([rmse_df, kriging_temp_err], ignore_index=True)

### IDW PM 2.5

In [122]:
pm2_5_idw_fc = 'pm2_5_idw_cross_valid'
pm2_5_idw_path = os.path.join(save_path_gdb, pm2_5_idw_fc)
arcpy.ga.CrossValidation(
    in_geostat_layer="pm2_5_idw_stats",
    out_point_feature_class=pm2_5_idw_fc
)
pm2_5_idw_out_fc = "PM2_5_IDW_Join"
arcpy.analysis.SpatialJoin(
    target_features="stations_XYTableToPoint",
    join_features="pm2_5_idw_cross_valid",
    out_feature_class=pm2_5_idw_out_fc,
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_COMMON",
    field_mapping='OID_ "OID_" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.OID_,-1,-1;sensor_index "sensor_index" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.sensor_index,-1,-1;last_modified "last_modified" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.last_modified,-1,-1;date_created "date_created" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.date_created,-1,-1;last_seen "last_seen" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.last_seen,-1,-1;name "name" true true false 100 Text 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.name,0,100;location_type "location_type" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.location_type,-1,-1;firmware_version "firmware_version" true true false 30 Text 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.firmware_version,0,30;uptime "uptime" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.uptime,-1,-1;position_rating "position_rating" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.position_rating,-1,-1;latitude "latitude" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.latitude,-1,-1;longitude "longitude" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.longitude,-1,-1;altitude "altitude" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.altitude,-1,-1;channel_state "channel_state" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.channel_state,-1,-1;channel_flags "channel_flags" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,stations_XYTableToPoint.channel_flags,-1,-1;OID1 "OID_" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.OID_,-1,-1;sensor_index_1 "sensor_index" true true false 4 Long 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.sensor_index,-1,-1;timestamp "timestamp" true true false 8 Date 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.timestamp,-1,-1;humidity "humidity" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.humidity,-1,-1;temperature "temperature" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.temperature,-1,-1;pressure "pressure" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.pressure,-1,-1;pm2_5 "pm2_5" true true false 8 Double 0 0,First,#,stations_XYTableToPoint,purpleair_historic.csv.pm2_5,-1,-1;Measured "Measured" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,Measured,-1,-1;Predicted "Predicted" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,Predicted,-1,-1;Error "Error" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,Error,-1,-1;StdError "Standard Error" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,StdError,-1,-1;Stdd_Error "Standardized Error" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,Stdd_Error,-1,-1;NormValue "Normal Value" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,NormValue,-1,-1;CRPS "Continuous Ranked Probability Score" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,CRPS,-1,-1;QuanVal "Validation Quantile" true true false 8 Double 0 0,First,#,pm2_5_kriging_cross_valid,QuanVal,-1,-1;Interval90 "Inside 90 Percent Interval" true true false 255 Text 0 0,First,#,pm2_5_kriging_cross_valid,Interval90,0,255;Interval95 "Inside 95 Percent Interval" true true false 255 Text 0 0,First,#,pm2_5_kriging_cross_valid,Interval95,0,255;Source_ID "Source ID" true false false 4 Long 0 0,First,#,pm2_5_kriging_cross_valid,Source_ID,-1,-1;Included "Included" true true false 255 Text 0 0,First,#,pm2_5_kriging_cross_valid,Included,0,255',
    match_option="INTERSECT",
    search_radius=None,
    distance_field_name=""
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\PM2_5_IDW_Join'>

In [125]:
input = pm2_5_idw_out_fc
arr_pm2_5_idw = arcpy.da.TableToNumPyArray(input, ("OBJECTID", "latitude", "longitude", "Error"))
idw_pm2_5_arr = pd.DataFrame(arr_pm2_5_idw, columns=["latitude", "Error"])
idw_pm2_5_err = idw_pm2_5_arr.rename(columns={"Error": "PM2_5_IDW_Error"})
rmse_df = rmse_df.merge(idw_pm2_5_arr, on="latitude", how="outer")

### Local Polynomial PM 2.5

In [48]:
pm2_5_poly_fc = 'pm2_5_poly_cross_valid'
pm2_5_poly_path = os.path.join(save_path_gdb, pm2_5_poly_fc)
arcpy.ga.CrossValidation(
    in_geostat_layer="pm2_5_poly_stats",
    out_point_feature_class=pm2_5_poly_fc
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\pm2_5_poly_cross_valid'>

### Kriging Temperature

In [49]:
temp_kriging_fc = 'temp_kriging_cross_valid'
temp_kriging_path = os.path.join(save_path_gdb, temp_kriging_fc)
arcpy.ga.CrossValidation(
    in_geostat_layer="temp_kriging_stats",
    out_point_feature_class=temp_kriging_fc
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\temp_kriging_cross_valid'>

### IDW Temprature

In [50]:
temp_idw_fc = 'temp_idw_cross_valid'
temp_idw_path = os.path.join(save_path_gdb, temp_idw_fc)
arcpy.ga.CrossValidation(
    in_geostat_layer="temp_idw_stats",
    out_point_feature_class=temp_idw_fc
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\temp_idw_cross_valid'>

### Local Polynomial Temperature

In [51]:
temp_poly_fc = 'temp_poly_cross_valid'
temp_poly_path = os.path.join(save_path_gdb, temp_poly_fc)
arcpy.ga.CrossValidation(
    in_geostat_layer="temp_poly_stats",
    out_point_feature_class=temp_poly_fc
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\temp_poly_cross_valid'>

## Calculate RMSE

## Raster to Point each Interpolation

## Clipping Each Interpolation

### Kriging PM 2.5 Clip

In [ ]:
out_clip_kriging_raster = "Kriging_clip_Apr1"
arcpy.management.Clip(
    in_raster=out_kriging_raster,
    rectangle="-93.32910837 44.8905885090001 -93.194328522 45.0512462900001",
    out_raster=out_clip_kriging_raster,
    in_template_dataset='mpls_boundary',
    nodata_value="3.4e+38",
    clipping_geometry="NONE",
    maintain_clipping_extent="NO_MAINTAIN_EXTENT"
)

### IDW PM 2.5 Clip

In [ ]:
out_clip_idw_raster = "IDW_clip_Apr1"
arcpy.management.Clip(
    in_raster=out_idw_raster,
    rectangle="-93.32910837 44.8905885090001 -93.194328522 45.0512462900001",
    out_raster=out_clip_idw_raster,
    in_template_dataset='mpls_boundary',
    nodata_value="3.4e+38",
    clipping_geometry="NONE",
    maintain_clipping_extent="NO_MAINTAIN_EXTENT"
)

### Local Polynomial PM 2.5 Clip

In [ ]:
out_clip_poly_raster = "Poly_clip_Apr1"
arcpy.management.Clip(
    in_raster=out_poly_raster,
    rectangle="-93.32910837 44.8905885090001 -93.194328522 45.0512462900001",
    out_raster=out_clip_poly_raster,
    in_template_dataset='mpls_boundary',
    nodata_value="3.4e+38",
    clipping_geometry="NONE",
    maintain_clipping_extent="NO_MAINTAIN_EXTENT"
)

### Kriging Temperature Clip

In [ ]:
out_clip_kriging_temp_raster = "Kriging_clip_temp_Apr1"
arcpy.management.Clip(
    in_raster=out_kriging_temp_raster,
    rectangle="-93.32910837 44.8905885090001 -93.194328522 45.0512462900001",
    out_raster=out_clip_kriging_temp_raster,
    in_template_dataset='mpls_boundary',
    nodata_value="3.4e+38",
    clipping_geometry="NONE",
    maintain_clipping_extent="NO_MAINTAIN_EXTENT"
)

### IDW Temperature Clip

In [ ]:
out_clip_idw_temp_raster = "IDW_clip_temp_Apr1"
arcpy.management.Clip(
    in_raster=out_idw_temp_raster,
    rectangle="-93.32910837 44.8905885090001 -93.194328522 45.0512462900001",
    out_raster=out_clip_idw_temp_raster,
    in_template_dataset='mpls_boundary',
    nodata_value="3.4e+38",
    clipping_geometry="NONE",
    maintain_clipping_extent="NO_MAINTAIN_EXTENT"
)

### Local Polynomial Clip

In [ ]:
out_clip_poly_temp_raster = "Poly_clip_temp_Apr1"
arcpy.management.Clip(
    in_raster=out_poly_temp_raster,
    rectangle="-93.32910837 44.8905885090001 -93.194328522 45.0512462900001",
    out_raster=out_clip_poly_temp_raster,
    in_template_dataset='mpls_boundary',
    nodata_value="3.4e+38",
    clipping_geometry="NONE",
    maintain_clipping_extent="NO_MAINTAIN_EXTENT"
)